In [30]:
xls.prep <- function(formula,data){
    
    matrix.baked <- stats::model.matrix(formula,data)
    
    df.baked <- base::as.data.frame(matrix.baked)
    
    base::rm(matrix.baked)
    
    variable_names <- base::names(df.baked)
    
    squared_names <- base::paste0(variable_names,'_squared')
    
    for(i in 1:base::length(squared_names)){
        df.baked[[squared_names[i]]] <- df.baked[[variable_names[i]]]**2
    }
    
    df.baked
}

In [31]:
xls.fit <- function(formula,
                    data,
                    lag_level=1,
                    lag_column = NULL,
                    error_weights = NULL,
                    error_ahead_level=4){
    
    if(base::nrow(data) < error_ahead_level){
        
        base::stop('The number of observations must be greater than error ahead level.')
        
    }
    
    
    dependent_var <- base::all.vars(formula)[1]
    independent_var <- base::all.vars(formula)[-1]
    
    if(base::is.null(lag_column)){
        
        lag_column <- dependent_var
        
    }
    
    if(base::is.null(error_weights)){
        
        dummy_weights <- base::seq(from = 0,to = 1,length.out = error_ahead_level + 1)
        error_weights <- dummy_weights[-1]/base::sum(dummy_weights[-1])
        base::rm(dummy_weights)
        
    }else if(base::length(error_weights) != error_ahead_level){
        
        base::stop('Error weights should have same length with ahead level.')
        
    }else if(base::sum(error_weights) != 1){
        
        base::stop('The sum of the error weights must be 1.')
        
    }
    
    #df <- base::as.data.frame(stats::model.matrix(formula,data))
    
}